# Test de lecture et conversion des coordonnées de l'ISS

Dans ce notebook, nous cherchons à visualiser les positions successives de l'ISS dans une représentation en trois dimensions autour de la Terre, à partir des données du fichier NASA.

## Contenu des données sources

In [1]:
import pandas as pd
# data from ISS.OEM_J2K_EPH.txt
df = pd.read_csv("ISS.OEM_J2K_EPH.txt", skiprows=41,
                 sep=" ", names=['datetime', 'x', 'y', 'z', 'vx', 'vy', 'vz'],
                 parse_dates=[0], infer_datetime_format=True)
# Présentation du contenu du fichier NASA
df.head()

,datetime,x,y,z,vx,vy,vz
0,2023-03-24 18:41:29.004,4482.285420,-1945.879050,-4735.454317,5.092194,4.987269,2.771175
1,2023-03-24 18:45:29.004,5527.582022,-693.041910,-3906.729727,3.565402,5.389738,4.093092
2,2023-03-24 18:49:29.004,6172.791603,609.934836,-2794.459786,1.778186,5.402386,5.119574
3,2023-03-24 18:53:29.004,6370.538918,1868.675170,-1479.074801,-0.140821,5.023142,5.775192
4,2023-03-24 18:57:29.004,6105.797591,2991.735742,-56.034355,-2.052317,4.278463,6.011005


In [2]:
import numpy as np
# Rayon terrestre moyen volumétrique
earth_radius = 6371
iss_altitude = np.sqrt(df['x']**2+df['y']**2+df['z']**2) - earth_radius
# Altitude de l'ISS
iss_altitude.describe()

count    5961.000000
mean      425.661024
std         5.256482
min       414.646859
25%       421.027759
50%       426.604911
75%       430.295824
max       433.795420
dtype: float64

Avec une altitude moyenne d'environ 425km, les positions de l'ISS apparaitront au plus près de la Terre, modélisée comme une sphère de 6371km de rayon.

## Création d'une fonction de visualisation pour la bibliothèque

Pour permettre des visualisations simples de modèles en trois dimensions, nous définissons une fonction `orbital_visualization` pour notre bibliothèque.

Cette fonction est définie en tant que `contextmanager`, permettant son utilisation avec `with`.

In [9]:
from contextlib import contextmanager

import numpy as np
import plotly.graph_objects as go

@contextmanager
def orbital_visualization():
    """Représentation d'un globle terrestre pour un affichage 3D de positions orbitales."""
    # Rayon terrestre moyen volumétrique
    earth_radius = 6371
    # Découpage en 24 méridiens et 60 parallèles
    u, v = np.mgrid[0:2 * np.pi:24j, 0:np.pi:60j]
    # Surface de la Terre
    x, y, z = np.array([np.cos(u) * np.sin(v), np.sin(u) * np.sin(v), np.cos(v)]) * earth_radius
    # Représentation des méridiens
    meridians = [go.Scatter3d(x=xx, y=yy, z=zz, line_width=1, line_color="cadetblue",
                              opacity=1, showlegend=False, hoverinfo='skip')
                 for xx, yy, zz in zip(x,y,z)]
    # Représentation de la surface de la Terre
    surface = go.Surface(x=earth_radius * np.cos(u)*np.sin(v),
                         y=earth_radius * np.sin(u)*np.sin(v),
                         z=earth_radius * np.cos(v),
                         opacity=0.7, colorscale="Blues_r", showscale=False, hoverinfo='none')
    # Représentation de l'axe de rotation (pour une meilleure compréhension du rendu)
    axis = go.Scatter3d(x=[0]*2, y=[0]*2, z=[-1.2*earth_radius, 1.2*earth_radius], line_width=2,
                        line_color='gray', opacity=0.6, showlegend=False)
    # Chargement de la représentation de la Terre comme figure de base
    fig = go.Figure(meridians + [axis, surface])
    # Template pour afficher les scatter3d en mode ligne par défaut
    orbit_template = go.layout.Template()
    orbit_template.data.scatter3d = [go.Scatter3d(mode="lines", line_width=3, hoverinfo='skip')]
    # Supression des plans d'axes en arrière-plan
    fig.update_scenes(xaxis_visible=False, yaxis_visible=False,zaxis_visible=False)
    # Paramètrage d'une caméra au plus près de la Terre
    fig.update_layout(scene_camera=dict(up=dict(x=0, y=np.sin(.409), z=np.cos(.409)), eye=dict(x=1.25, y=0, z=0)),
                      scene_dragmode='orbit', margin=dict(l=0,r=0,b=0,t=50), template=orbit_template)
    # Renvoi de la figure pour utilisation
    yield fig
    # Affichage de la représentation
    fig.show()

## Test de la fonction

Pour tester notre fonction, nous traçons la trajectoire de l'ISS avec l'ensemble des données de position du jeu de données.

### Dans le repère céleste

On représente les positions de l'ISS dans le repère céleste (le repère est centré sur le centre de la Terre et ne tourne pas avec elle).

In [10]:
with orbital_visualization() as fig:
    fig.add_scatter3d(x=df.x, y=df.y, z=df.z, mode="lines", showlegend=False,
                      line_color=df.index, line_colorscale="plasma", hoverinfo='skip')
    fig.update_layout(title="Position céleste")

### Dans le repère terrestre

On représente ces mêmes positions dans le repère terrestre (le repère est centré sur le centre de la Terre et tourne avec elle).

On définit la fonction `celestial2terrestrial` pour changer de repère, en s'appuyant sur la biblothèque `astropy`.

In [13]:
from astropy.coordinates import SkyCoord, ITRS, GCRS
from astropy import units as u

def celestial2terrestrial(x, y, z, datetime, mode='cartesian'):
    """Transformation de coordonnées depuis le repère celeste vers le repère terrestre.
    
    La fonction prend en paramètre les coordonnées cartésiennes d'origine ainsi que la
    date et l'heure de la position, nécessaire pour la projeter dans le repère terrestre.
    La fonction retourne les coordonnées (x,y,z) dans le repère terrestre,
    ou les valeurs (longitude, latitude, distance) si le paramètre facultatif `mode='spherical'`
    est utilisé.
    """
    # coordonnées sidérales depuis les données
    geo = SkyCoord(x=x, y=y, z=z, unit=u.km, frame=GCRS,
                   representation_type='cartesian',
                   obstime=datetime).transform_to(ITRS)
    # Représentation cartésienne
    if mode == 'cartesian':
        return geo.x.value, geo.y.value, geo.z.value
    # Représentation sphérique
    elif mode == 'spherical':
        sph = geo.represent_as('spherical')
        return sph.lon.degree, sph.lat.degree, sph.distance.value
    # Paramètre de représentation incorrect
    else:
        raise ValueError("mode should be either 'cartesian' or 'spherical'.")

df['x2'], df['y2'], df['z2'] = df.apply(lambda row: celestial2terrestrial(row.x, row.y, row.z, row.datetime),
                                        axis=1, result_type='expand').transpose().values

In [14]:
with orbital_visualization() as fig:
    fig.add_scatter3d(x=df.x2, y=df.y2, z=df.z2, mode="lines", showlegend=False,
                      line_color=df.index, line_colorscale="plasma", hoverinfo='skip'),
    fig.update_layout(title="Position terrestre")